In [1]:
import os
from datasets import load_dataset
from tqdm import tqdm 

In [2]:
base_dir = "./data_vi_culturax"
output_dir = "./data_vi_culturax_converted"

In [3]:
for filename in tqdm(os.listdir(base_dir)): 
    file_path = os.path.join(base_dir, filename)

    dataset = load_dataset("parquet", data_files=file_path)
    dataset = dataset.remove_columns(["timestamp", "url", "source"])

    dataset.save_to_disk("data_vi_culturax_converted")


  0%|          | 0/90 [00:00<?, ?it/s]

Saving the dataset (0/6 shards):   0%|          | 0/640072 [00:00<?, ? examples/s]

  1%|          | 1/90 [00:03<05:33,  3.74s/it]

Saving the dataset (0/6 shards):   0%|          | 0/640077 [00:00<?, ? examples/s]

  2%|▏         | 2/90 [00:09<06:50,  4.67s/it]


KeyboardInterrupt: 

In [3]:
dataset = load_dataset("./data_vi_culturax", streaming=True)

Resolving data files:   0%|          | 0/90 [00:00<?, ?it/s]

In [49]:
import pyarrow as pa
import pyarrow.feather as feather 

content = []
output_file_counter = 36 

for i, row in tqdm(enumerate(dataset["train"])): 
    content.append(row["text"]) 

    if i > 36000000 and i % 1e6 == 0 and i != 0: 
        table = pa.Table.from_arrays([pa.array(content)], names=["text"])
        output_filepath = f"./data_vi_culturax_converted/{str(output_file_counter).zfill(5)}.arrow"

        with pa.CompressedOutputStream(output_filepath, compression="gzip") as stream: 
            writer = pa.RecordBatchStreamWriter(stream, table.schema)
            writer.write_table(table) 
            writer.close()

        # feather.write_feather(table, output_filepath , version=1)

        output_file_counter += 1
        content = []

4855107it [07:25, 12683.20it/s]

In [42]:
if content: 
    table = pa.Table.from_arrays([pa.array(content)], names=["text"])
    output_filepath = f"./data_vi_culturax_converted/{str(output_file_counter + 1).zfill(5)}.arrow"

    with pa.output_stream(output_filepath) as stream: 
        writer = pa.RecordBatchStreamWriter(stream, table.schema)
        writer.write_table(table) 
        writer.close()

In [47]:
test_dataset = load_dataset("arrow", data_files="./data_vi_culturax_converted/00000.arrow")

Generating train split: 0 examples [00:00, ? examples/s]